### 부동산 데이터 전처리

In [112]:
import pandas as pd
import numpy as np
from haversine import haversine, haversine_vector
from geopy.geocoders import Nominatim
import ast



In [113]:
apartment = pd.read_csv("청약홈_합본.csv", encoding='cp949')

1. 대형건설사
* '대형건설사' 기준: 10대 건설사(삼성물산, 현대건설, DL이앤씨, 포스코이앤씨, GS건설, 대우건설, 현대엔지니어링, 롯데건설, SK에코플랜트, HDC현대산업개발)에 포함

In [114]:
apartment['건설사'] = apartment['건설사'].str.replace("[0-9.\s]|\t|\(주\)|㈜|주\)|주식회사", "", regex=True)

In [115]:
def replace_firm_name(firm):
    if '현대산업개발' in firm:
        firm = firm.replace('HDC현대산업개발', '현대산업개발')
        firm = firm.replace('에이치디씨현대산업개발', '현대산업개발')
        firm = firm.replace('현대산업개발', 'HDC현대산업개발')
    if '디엘이앤씨' in firm:
        firm = firm.replace('디엘이앤씨', 'DL이앤씨') 
    if '지에스건설' in firm:
        firm = firm.replace('지에스건설', 'GS건설')
    if '에스케이에코플랜트' in firm:
        firm = firm.replace('에스케이에코플랜트', 'SK에코플랜트')
    if '에스케이건설' in firm:
        firm = firm.replace('에스케이건설', 'SK에코플랜트') # 사명 변경
    if '포스코건설' in firm:
        firm = firm.replace('포스코건설', '포스코이앤씨')   # 사명 변경
    
    return firm

apartment['건설사'] = apartment['건설사'].apply(replace_firm_name)
apartment['건설사'].unique()

array(['롯데건설', 'SK에코플랜트,HDC현대산업개발', '현대엔지니어링', '동양건설산업', 'GS건설', '원건설',
       '성우에스아이', '삼성물산', '스위트드림종합건설', '덕인컨스텍', '대우건설', '현대건설', '혜림건설',
       '중흥토건', '한신공영', '한울건설산업', '호반건설', 'HDC현대산업개발', '우석종합건설', '제일건설',
       '대림산업', '일신건영', '대원', '라인건설', 'DL건설,DL이앤씨', '현진에버빌', '대광건영',
       '호반산업', '대방건설', '일군토건', '에이치디씨아이앤콘스',
       '대림산업,금호산업,계룡건설산업,고덕종합건설,금성백조건설', '한화건설', '까뮤이앤씨', '하호종합건설',
       '신한종합건설', '금호건설', '나경종합건설', '신동아건설', '케이씨씨건설', '에쓰와이이앤씨', '쌍용건설',
       '한라', '대우산업개발', '금강주택', '대우건설,금호산업,태영건설', 'GS건설,대우건설', '삼부토건',
       'HDC현대산업개발,롯데건설', '유림E&C', '대우건설,SK에코플랜트', '양우건설', '천명토건외',
       '경남기업,티케이케미칼', '영무건설,하가건설', '승윤종합건설,길성종합건설', '포스코이앤씨', '진산건설',
       '신원종합개발', '우미개발,우미산업개발', '대성베르힐건설,디에스종합건설', '서희건설', '한국토지신탁,이랜드건설',
       '현대비에스앤씨', '한양,보성산업', '리젠시빌주택', '한진중공업,에이치에스공영,해원산업', '동원개발',
       'SK에코플랜트,한진중공업', '우미건설', '일성건설', 'GS건설,쌍용건설', '경남기업', 'SK에코플랜트',
       '이수건설', '이테크건설', '디에스종합건설,대성베르힐건설', '우미개발,우미토건', '화성개발', '동우개발',
       '삼미건설', '남광토건,금광기업', '자이에스앤디'

In [116]:
대형건설사 = ['삼성물산', '현대건설', 'DL이앤씨', '포스코이앤씨', 'GS건설', '대우건설', '현대엔지니어링', '롯데건설', 'SK에코플랜트', 'HDC현대산업개발']
apartment['대형건설사'] = apartment['건설사'].apply(lambda x: 'YES' if any(keyword in x for keyword in 대형건설사) else 'NO')

In [117]:
apartment.drop(['Unnamed: 0'], axis=1, inplace=True)
apartment['location'] = tuple(zip(apartment['위도'], apartment['경도']))
apartment_location = list(apartment['location'])

2. 가장 가까운 지하철역명과 그 역과의 거리

In [118]:
all_rail = pd.read_excel("전체_도시철도역사정보_20230630.xlsx")

In [119]:
cities_to_remove = ['대구', '부산', '대전', '광주광역시', '울산'] # '광주'만 넣을 경우 '경기도 광주' 포함
greater_seoul_rail = ~(all_rail['운영기관명'].str.contains('|'.join(cities_to_remove)) | all_rail['역사도로명주소'].str.contains('|'.join(cities_to_remove)))
rail = all_rail[greater_seoul_rail]

In [120]:
rail.loc[:, '역사명'] = rail['역사명'].str.replace(r"\s|\n|\([^()]*\)|[.·]", "", regex=True)
rail.loc[:, '역사명'] = rail['역사명'].apply(lambda x: x + '역' if not x.endswith('역') else x)

In [121]:
# station 데이터로 보완
station = pd.read_csv("station_coordinate.csv")
station['name'] = station['name'] + '역'
station['name'] = station['name'].str.replace("[.,]", "", regex=True)
station_name = set(station['name'].unique())

rail_name = set(rail['역사명'].unique())

In [122]:
list(station_name - rail_name)

['이수역', '원곡역']

In [123]:
# 총신대입구(이수)역은 4호선과 7호선 2개 역
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '4호선'), '역위도'] = station[station['name'] == '총신대입구역']['lat'].values
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '4호선'), '역경도'] = station[station['name'] == '총신대입구역']['lng'].values
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '7호선'), '역위도'] = station[station['name'] == '이수역']['lat'].values
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '7호선'), '역경도'] = station[station['name'] == '이수역']['lng'].values

In [124]:
# '원곡역'은 '시우역'으로 역사명이 변경되었음
rail[rail['역사명'] == '시우역']

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
1041,4814,시우역,I41WS,서해선,Siu,時雨,일반역,NaN,NaN,37.313767,126.798303,한국철도공사,경기도 안산시 단원구 동산로 지하 50,1544-7788,2022-08-31


In [125]:
rail = rail[rail['노선명'] != '자기부상철도'] # 인천공항 자기부상열차는 무기한 운행 중단

In [126]:
merged_rail = rail.merge(station[['name', 'lat', 'lng']], left_on = '역사명', right_on='name', how='left')

merged_rail['역위도'] = merged_rail['역위도'].fillna(merged_rail['lat'])
merged_rail['역경도'] = merged_rail['역경도'].fillna(merged_rail['lng'])

merged_rail.drop(['name', 'lat', 'lng'], axis=1, inplace=True)
merged_rail = merged_rail.drop_duplicates()

In [127]:
merged_rail.head()

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
0,3110,계양역,S2801,인천지하철 1호선,Gyeyang,桂陽,환승역,S2801+I28A1,공항철도,37.571539,126.736319,인천교통공사,인천광역시 계양구 다남로 24,032-710-9105,2022-04-30 00:00:00
2,3111,귤현역,S2801,인천지하철 1호선,Gyulhyeon,橘峴,일반역,-,-,37.566362,126.742498,인천교통공사,인천광역시 계양구 장제로 1136,032-515-9104,2022-04-30 00:00:00
3,3112,박촌역,S2801,인천지하철 1호선,Bakchon,朴村,일반역,-,-,37.553525,126.744946,인천교통공사,인천광역시 계양구 장제로 992,032-519-3122,2022-04-30 00:00:00
4,3113,임학역,S2801,인천지하철 1호선,Imhak,林鶴,일반역,-,-,37.545058,126.738642,인천교통공사,인천광역시 계양구 장제로 875,032-541-3113,2022-04-30 00:00:00
5,3114,계산역,S2801,인천지하철 1호선,Gyesan,桂山,일반역,-,-,37.543243,126.728436,인천교통공사,인천광역시 계양구 경명대로 1089,032-546-3151,2022-04-30 00:00:00


In [128]:
# 여전히 위경도가 없음  
blank_df = merged_rail[merged_rail['역위도'].isna() | merged_rail['역경도'].isna()]

In [129]:
def geocoding(address):
    geo_local = Nominatim(user_agent='South Korea')
    location = geo_local.geocode(address)
    if location is not None and location.latitude is not None and location.longitude is not None:
        geo = (location.latitude, location.longitude)
        return geo
    else:
        return (0, 0)
    
blank_df.loc[:, '역위도'] = blank_df['역사도로명주소'].apply(lambda x: geocoding(x)[0])
blank_df.loc[:, '역경도'] = blank_df['역사도로명주소'].apply(lambda x: geocoding(x)[1])


In [130]:
# 여전히 업데이트 되지 않는 역들
blank_df[(blank_df['역위도'] == 0)|(blank_df['역경도'] == 0)][['역사명', '역사도로명주소']]

,역사명,역사도로명주소
75,산곡역,인천광역시 부평구 길주로 지하 379 (산곡동)
186,오남역,경기도 남양주시 오남읍 진건오남로 지하 929
453,강일역,서울특별시 강동구 고덕로 지하456(강일동)
454,미사역,경기도 하남시 미사강변동로 지하90(망월동)
455,하남풍산역,경기도 하남시 덕풍서로 지하50(덕풍동)
456,하남시청역,경기도 하남시 하남대로 지하820(덕풍동)
457,하남검단산역,경기도 하남시 대청로 지하100(창우동)
587,남위례역,경기도 성남시 수정구 공원로 601(복정동)
637,삼전역,서울특별시 송파구 백제고분로 지하 187
638,석촌고분역,서울특별시 송파구 삼학사로 지하 53


In [131]:
# geocoding 함수가 적용되지 않는 것들은 데이터상 주소의 문제 -> 주소 수정

blank_df.loc[blank_df['역사명'] == '산곡역', '역사도로명주소'] = '인천광역시 부평구 산곡동 10-32'
blank_df.loc[blank_df['역사명'] == '오남역', '역사도로명주소'] = '경기도 남양주시 오남읍 진건오남로 929'
blank_df.loc[blank_df['역사명'] == '강일역', '역사도로명주소'] = '서울특별시 강동구 강일동 산22-14'
blank_df.loc[blank_df['역사명'] == '미사역', '역사도로명주소'] = '경기도 하남시 망월동 109-8'
blank_df.loc[blank_df['역사명'] == '하남풍산역', '역사도로명주소'] = '경기도 하남시 덕풍동 727-3'
blank_df.loc[blank_df['역사명'] == '하남시청역', '역사도로명주소'] = '경기도 하남시 신장동 510-2'
blank_df.loc[blank_df['역사명'] == '하남검단산역', '역사도로명주소'] = '경기도 하남시 창우동 526'
blank_df.loc[blank_df['역사명'] == '남위례역', '역사도로명주소'] = '경기도 성남시 수정구 복정동 57'
blank_df.loc[blank_df['역사명'] == '삼전역', '역사도로명주소'] = '서울특별시 송파구 잠실동 347'
blank_df.loc[blank_df['역사명'] == '석촌고분역', '역사도로명주소'] = '서울특별시 송파구 석촌동 157'
blank_df.loc[blank_df['역사명'] == '송파나루역', '역사도로명주소'] = '서울특별시 송파구 송파동 3'
blank_df.loc[blank_df['역사명'] == '한성백제역', '역사도로명주소'] = '서울특별시 송파구 방이동 88-17'
blank_df.loc[blank_df['역사명'] == '둔촌오륜역', '역사도로명주소'] = '서울특별시 강동구 강동대로 303'
blank_df.loc[blank_df['역사명'] == '중앙보훈병원역', '역사도로명주소'] = '서울특별시 강동구 둔촌동 8-1'


In [132]:
blank_df.loc[:, '역위도'] = blank_df['역사도로명주소'].apply(lambda x: geocoding(x)[0])
blank_df.loc[:, '역경도'] = blank_df['역사도로명주소'].apply(lambda x: geocoding(x)[1])
blank_df.head()

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
75,3762,산곡역,S1107,도시철도 7호선,Sangok,山谷,일반역,-,-,37.499181,126.702419,인천교통공사,인천광역시 부평구 산곡동 10-32,032-451-1393,2022-04-30 00:00:00
172,G100,양촌역,L41G1,김포골드라인,Yangchon,阳村,일반역,NaN,NaN,37.639058,126.614692,김포골드라인운영㈜,경기도 김포시 양촌역길 107,031-8048-1700,2023-05-26 00:00:00
173,G101,구래역,L41G1,김포골드라인,Gurae,旧来,일반역,NaN,NaN,37.646493,126.628492,김포골드라인운영㈜,경기도 김포시 김포한강7로 87,031-8048-1710,2023-05-26 00:00:00
174,G102,마산역,L41G1,김포골드라인,Masan,马山,일반역,NaN,NaN,37.635342,126.671371,김포골드라인운영㈜,경기도 김포시 김포한강3로 442,031-8048-1720,2023-05-26 00:00:00
175,G103,장기역,L41G1,김포골드라인,Janggi,象棋,일반역,NaN,NaN,37.640823,126.666295,김포골드라인운영㈜,경기도 김포시 김포한강1로 59,031-8048-1730,2023-05-26 00:00:00


In [133]:
# blank_df값으로 merged_rail 업데이트

for index, data in merged_rail.iterrows():
    if data['역사명'] in blank_df['역사명'].values:
        new_info = blank_df.loc[blank_df['역사명'] == data['역사명']]
        merged_rail.at[index, '역위도'] = new_info['역위도'].values[0]
        merged_rail.at[index, '역경도'] = new_info['역경도'].values[0]

In [134]:
merged_rail.loc[(merged_rail['역위도'] == 0) | (merged_rail['역경도'] == 0)]

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자


In [135]:
merged_rail['노선명'].unique()

array(['인천지하철 1호선', '인천지하철 2호선', '도시철도 7호선', '에버라인', '인천국제공항선', '우이신설선',
       '신분당선', '수도권 경량도시철도 신림선', '김포골드라인', '진접선', '1호선', '2호선', '3호선',
       '4호선', '5호선', '6호선', '7호선', '8호선', '수도권  도시철도 9호선', '의정부', '경강선',
       '경부선', '경원선', '경의중앙선', '경인선', '경춘선', '분당선', '수인선', '안산과천선', '일산선',
       '서해선'], dtype=object)

In [136]:
merged_rail.loc[merged_rail['노선명'] == '도시철도 7호선', '노선명'] = '7호선'
merged_rail.loc[merged_rail['노선명'] == '수도권  도시철도 9호선', '노선명'] = '9호선'
merged_rail.loc[merged_rail['노선명'] == '의정부', '노선명'] = '의정부선'
merged_rail.loc[merged_rail['노선명'] == '일산선', '노선명'] = '3호선'
merged_rail.loc[merged_rail['노선명'] == '수도권 경량도시철도 신림선', '노선명'] = '신림선'
merged_rail.loc[merged_rail['노선명'] == '경부선', '노선명'] = '1호선'
merged_rail.loc[merged_rail['노선명'] == '경인선', '노선명'] = '1호선'
merged_rail.loc[merged_rail['노선명'] == '경원선', '노선명'] = '1호선'
merged_rail.loc[merged_rail['노선명'] == '안산과천선', '노선명'] = '4호선'

errored = merged_rail['노선명'] == '신분당선'
merged_rail.loc[errored, ['역위도', '역경도']] = merged_rail.loc[errored, ['역경도', '역위도']].values

In [137]:
merged_rail['노선명'].unique()

array(['인천지하철 1호선', '인천지하철 2호선', '7호선', '에버라인', '인천국제공항선', '우이신설선',
       '신분당선', '신림선', '김포골드라인', '진접선', '1호선', '2호선', '3호선', '4호선', '5호선',
       '6호선', '8호선', '9호선', '의정부선', '경강선', '경의중앙선', '경춘선', '분당선', '수인선',
       '서해선'], dtype=object)

In [138]:
# '역사명'이 다른데 '역위도'와 '역경도'가 같은 행 -> 주소 수정 필요

duplicated = merged_rail[merged_rail.duplicated(subset=['역위도', '역경도'], keep=False) & 
                         ~merged_rail.duplicated(subset=['역위도', '역경도', '역사명'], keep=False)
                        ]
duplicated_sorted = duplicated.sort_values(by=['역위도'])
duplicated_sorted

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
716,1701,구로역,I4101,1호선,Guro,九 老,일반역,NaN,NaN,37.503178,126.882037,한국철도공사,서울시 구로구 구로중앙로 174,1544-7788,2022-08-31
814,1205,구리역,I4108,경의중앙선,Guri,九 里,일반역,NaN,NaN,37.503178,126.882037,한국철도공사,경기도 구리시 건원대로 34번길 32-29(인창동 244-2),1544-7788,2022-08-31
177,G105,걸포북변역,L41G1,김포골드라인,Geolpo Bukbyeon,杰浦北边,일반역,NaN,NaN,37.631537,126.708785,김포골드라인운영㈜,경기도 김포시 김포대로 1040,031-8048-1750,2023-05-26 00:00:00
178,G106,사우역,L41G1,김포골드라인,Sau(Gimpo City Hall),萨乌,일반역,NaN,NaN,37.631537,126.708785,김포골드라인운영㈜,경기도 김포시 김포대로 852,031-8048-1760,2023-05-26 00:00:00
179,G107,풍무역,L41G1,김포골드라인,Pungmu,风舞,일반역,NaN,NaN,37.631537,126.708785,김포골드라인운영㈜,경기도 김포시 김포대로 710,031-8048-1770,2023-05-26 00:00:00
784,1268,화전역,I4108,경의중앙선,Hwajeon,花 田,일반역,NaN,NaN,37.637837,126.832503,한국철도공사,경기도 고양시 덕양구 화랑로 53(화전동 183-10),1544-7788,2022-08-31
1016,1952,화정역,I4106,3호선,Hwajeong,花 井,일반역,NaN,NaN,37.637837,126.832503,한국철도공사,경기도 고양시 덕양구 화정동 400-2,1544-7788,2022-08-31
175,G103,장기역,L41G1,김포골드라인,Janggi,象棋,일반역,NaN,NaN,37.640823,126.666295,김포골드라인운영㈜,경기도 김포시 김포한강1로 59,031-8048-1730,2023-05-26 00:00:00
176,G104,운양역,L41G1,김포골드라인,Unyang,云阳,일반역,NaN,NaN,37.640823,126.666295,김포골드라인운영㈜,경기도 김포시 김포한강1로 235,031-8048-1740,2023-05-26 00:00:00


In [139]:
merged_rail.loc[merged_rail['역사명'] == '구로역', '역위도'] = geocoding(merged_rail[merged_rail['역사명']=='구로역']['역사도로명주소'].values[0])[0]
merged_rail.loc[merged_rail['역사명'] == '구로역', '역경도'] = geocoding(merged_rail[merged_rail['역사명']=='구로역']['역사도로명주소'].values[0])[1]

merged_rail.loc[merged_rail['역사명'] == '구리역', '역사도로명주소'] = '경기도 구리시 인창동 244-2'
merged_rail.loc[merged_rail['역사명'] == '구리역', '역위도'] = geocoding(merged_rail[merged_rail['역사명']=='구리역']['역사도로명주소'].values[0])[0]
merged_rail.loc[merged_rail['역사명'] == '구리역', '역경도'] = geocoding(merged_rail[merged_rail['역사명']=='구리역']['역사도로명주소'].values[0])[1]

merged_rail.loc[merged_rail['역사명'] == '걸포북변역', '역사도로명주소'] = '경기도 김포시 북변동 135-10'
merged_rail.loc[merged_rail['역사명'] == '걸포북변역', '역위도'] = geocoding(merged_rail[merged_rail['역사명']=='걸포북변역']['역사도로명주소'].values[0])[0]
merged_rail.loc[merged_rail['역사명'] == '걸포북변역', '역경도'] = geocoding(merged_rail[merged_rail['역사명']=='걸포북변역']['역사도로명주소'].values[0])[1]

merged_rail.loc[merged_rail['역사명'] == '사우역', '역사도로명주소'] = '경기도 김포시 사우동 854'
merged_rail.loc[merged_rail['역사명'] == '사우역', '역위도'] = geocoding(merged_rail[merged_rail['역사명']=='사우역']['역사도로명주소'].values[0])[0]
merged_rail.loc[merged_rail['역사명'] == '사우역', '역경도'] = geocoding(merged_rail[merged_rail['역사명']=='사우역']['역사도로명주소'].values[0])[1]

merged_rail.loc[merged_rail['역사명'] == '풍무역', '역사도로명주소'] = '경기도 김포시 김포대로 710'
merged_rail.loc[merged_rail['역사명'] == '풍무역', '역위도'] = geocoding(merged_rail[merged_rail['역사명']=='풍무역']['역사도로명주소'].values[0])[0]
merged_rail.loc[merged_rail['역사명'] == '풍무역', '역경도'] = geocoding(merged_rail[merged_rail['역사명']=='풍무역']['역사도로명주소'].values[0])[1]

merged_rail.loc[merged_rail['역사명'] == '화전역', '역사도로명주소'] = '경기도 고양시 덕양구 화전동 183-10'
merged_rail.loc[merged_rail['역사명'] == '화전역', '역위도'] = geocoding(merged_rail[merged_rail['역사명']=='화전역']['역사도로명주소'].values[0])[0]
merged_rail.loc[merged_rail['역사명'] == '화전역', '역경도'] = geocoding(merged_rail[merged_rail['역사명']=='화전역']['역사도로명주소'].values[0])[1]

merged_rail.loc[merged_rail['역사명'] == '화정역', '역사도로명주소'] = '경기도 고양시 덕양구 화정동 1098'
merged_rail.loc[merged_rail['역사명'] == '화정역', '역위도'] = geocoding(merged_rail[merged_rail['역사명']=='화정역']['역사도로명주소'].values[0])[0]
merged_rail.loc[merged_rail['역사명'] == '화정역', '역경도'] = geocoding(merged_rail[merged_rail['역사명']=='화정역']['역사도로명주소'].values[0])[1]

merged_rail.loc[merged_rail['역사명'] == '장기역', '역사도로명주소'] = '경기도 김포시 장기동 1791'
merged_rail.loc[merged_rail['역사명'] == '장기역', '역위도'] = geocoding(merged_rail[merged_rail['역사명']=='장기역']['역사도로명주소'].values[0])[0]
merged_rail.loc[merged_rail['역사명'] == '장기역', '역경도'] = geocoding(merged_rail[merged_rail['역사명']=='장기역']['역사도로명주소'].values[0])[1]

merged_rail.loc[merged_rail['역사명'] == '운양역', '역사도로명주소'] = '경기도 김포시 운양동 1403'
merged_rail.loc[merged_rail['역사명'] == '운양역', '역위도'] = geocoding(merged_rail[merged_rail['역사명']=='운양역']['역사도로명주소'].values[0])[0]
merged_rail.loc[merged_rail['역사명'] == '운양역', '역경도'] = geocoding(merged_rail[merged_rail['역사명']=='운양역']['역사도로명주소'].values[0])[1]

In [140]:
duplicated = merged_rail[merged_rail.duplicated(subset=['역위도', '역경도'], keep=False) & 
                         ~merged_rail.duplicated(subset=['역위도', '역경도', '역사명'], keep=False)
                        ]
duplicated

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자


In [141]:
merged_rail['location'] = tuple(zip(merged_rail['역위도'], merged_rail['역경도']))
merged_rail['subway'] = merged_rail['노선명'] + ' ' + merged_rail['역사명']

In [142]:
subway_dict = dict(zip(merged_rail['subway'], merged_rail['location']))
subway_location = list(subway_dict.values())
to_subway = haversine_vector(subway_location, apartment_location, comb=True)

close_subway = []
close_distance = []
for i in range(len(to_subway)):
    subway_index = int(np.argmin(to_subway[i]))
    closest_subway = list(subway_dict.keys())[subway_index]
    closest_distance = to_subway[i][subway_index]
    close_subway.append(closest_subway)
    close_distance.append(closest_distance)

In [143]:
apartment['지하철역'] = close_subway
apartment['지하철역_거리(km)'] = close_distance
apartment['지하철역_거리(km)'] = apartment['지하철역_거리(km)'].round(4)
apartment.head()

,아파트명,면적,법정동주소,도로명주소,위도,경도,세대수,임대세대수,영구임대세대수,국민임대세대수,...,현관구조,가격,초등학교_학군정보,초등학교_설립정보,입주예정일,공급액(만원),대형건설사,location,지하철역,지하철역_거리(km)
0,롯데캐슬 리버파크 시그니처(자양),84A㎡,서울시 광진구 자양동 236번지 일원,서울시 광진구,37.537863,127.062859,878,99.0,0.0,0.0,...,계단식,NaN,서울경수초등학교,혁신,2023-07-01,"81,100",YES,"(37.537863, 127.062859)",2호선 건대입구역,0.7221
1,롯데캐슬 리버파크 시그니처(자양),85B㎡,서울시 광진구 자양동 236번지 일원,서울시 광진구,37.537863,127.062859,878,99.0,0.0,0.0,...,계단식,NaN,서울경수초등학교,혁신,2023-07-01,"80,500",YES,"(37.537863, 127.062859)",2호선 건대입구역,0.7221
2,롯데캐슬 리버파크 시그니처(자양),111A㎡,서울시 광진구 자양동 236번지 일원,서울시 광진구,37.537863,127.062859,878,99.0,0.0,0.0,...,계단식,NaN,서울경수초등학교,혁신,2023-07-01,"106,900",YES,"(37.537863, 127.062859)",2호선 건대입구역,0.7221
3,롯데캐슬 리버파크 시그니처(자양),112B㎡,서울시 광진구 자양동 236번지 일원,서울시 광진구,37.537863,127.062859,878,99.0,0.0,0.0,...,계단식,NaN,서울경수초등학교,혁신,2023-07-01,"107,200",YES,"(37.537863, 127.062859)",2호선 건대입구역,0.7221
4,롯데캐슬 리버파크 시그니처(자양),112C㎡,서울시 광진구 자양동 236번지 일원,서울시 광진구,37.537863,127.062859,878,99.0,0.0,0.0,...,계단식,NaN,서울경수초등학교,혁신,2023-07-01,"103,300",YES,"(37.537863, 127.062859)",2호선 건대입구역,0.7221


3. 3km 반경 이내 의료기관 개수
* 3차 의료기관: '상급종합'
* 2차 의료기관: '종합병원', '병원', '한방병원', '치과병원'
* 1차 의료기관: '의원', '치과의원', '한의원', '보건소', '보건지소', '보건진료소', '보건의료원'

In [144]:
# hospital = pd.read_excel("전국병의원_202306.xlsx")
# hospital = hospital.loc[hospital['시도코드명'].isin(['서울', '경기', '인천']), ['요양기관명', '종별코드명', '시도코드명', '주소']]
# hospital.loc[hospital['종별코드명'].isin(['상급종합']), '종별코드명'] = '3차'
# hospital.loc[hospital['종별코드명'].isin(['종합병원', '병원', '한방병원', '치과병원']), '종별코드명'] = '2차'
# hospital.loc[hospital['종별코드명'].isin(['의원', '치과의원', '한의원', '보건소', '보건지소', '보건진료소', '보건의료원']), '종별코드명'] = '1차'
# hospital = hospital.loc[hospital['종별코드명'].isin(['1차', '2차', '3차'])]

# geocoding 함수를 바로 적용하니 너무 오래 걸리는 문제 -> Naver API를 이용하여 해결한 hospital.csv 파일 

hospital = pd.read_csv("hospital.csv")
hospital = hospital[hospital['location'] != '(0.0, 0.0)']
hospital.loc[:, 'location'] = hospital['location'].apply(ast.literal_eval)
hospital

,요양기관명,종별코드명,시도코드명,주소,location
0,의료법인한전의료재단 한일병원,2차,서울,"서울특별시 도봉구 우이천로 308, 한일병원 (쌍문동)","(37.6461401, 127.0291228)"
1,의료법인서울효천의료재단 에이치플러스양지병원,2차,서울,"서울특별시 관악구 남부순환로 1636, 양지병원 (신림동)","(37.4841537, 126.9325692)"
2,강남베드로병원,2차,서울,"서울특별시 강남구 남부순환로 2649, 베드로병원 (도곡동)","(37.4856196, 127.0396045)"
3,혜민병원,2차,서울,"서울특별시 광진구 자양로 85, (자양동)","(37.5354079, 127.0836246)"
4,서울특별시서북병원,2차,서울,"서울특별시 은평구 갈현로7길 49, (역촌동)","(37.60421, 126.9044499)"
...,...,...,...,...,...
39029,루나여성의원,1차,서울,"서울특별시 마포구 월드컵북로 402, 케이지아이티센터 204호, 205호 (상암동)","(37.5801078, 126.8889824)"
39030,서울정신건강의학과의원,1차,서울,"서울특별시 강남구 선릉로 517, 비젼빌딩 602호 (역삼동)","(37.5057184, 127.0481651)"
39031,서울엔(N)비뇨의학과의원,1차,서울,"서울특별시 강동구 올림픽로 660, 201~202호 (천호동, 천호엘크루주상복합)","(37.5401503, 127.1245234)"
39032,정릉손정신건강의학과의원,1차,서울,"서울특별시 성북구 보국문로 43, 3층 (정릉동)","(37.6081881, 127.0098725)"


In [145]:
hospital_location_1 = list(hospital[hospital['종별코드명'] == '1차']['location'].unique())
hospital_location_2 = list(hospital[hospital['종별코드명'] == '2차']['location'].unique())
hospital_location_3 = list(hospital[hospital['종별코드명'] == '3차']['location'].unique())

직선거리 1차 병원 0.5km 2차 병원 2km 3차 병원 3km

In [146]:
distance_to_hospitals = haversine_vector(hospital_location_1, apartment_location, comb=True)

num_close_hospital_1 = []
for distance in distance_to_hospitals:
    number = np.sum(distance < 0.5)
    num_close_hospital_1.append(number)
apartment['1차병원'] = num_close_hospital_1

In [147]:
distance_to_hospitals = haversine_vector(hospital_location_2, apartment_location, comb=True)

num_close_hospital_2 = []
for distance in distance_to_hospitals:
    number = np.sum(distance < 2)
    num_close_hospital_2.append(number)

apartment['2차병원'] = num_close_hospital_2

In [148]:
distance_to_hospitals = haversine_vector(hospital_location_3, apartment_location, comb=True)

num_close_hospital_3 = []
for distance in distance_to_hospitals:
    number = np.sum(distance < 3)
    num_close_hospital_3.append(number)
apartment['3차병원'] = num_close_hospital_3

4. 세대평균 주차대수

In [149]:
# 데이터 보충
apartment.loc[apartment['아파트명'] == '수유 시그니티', '주차대수'] = 42
apartment.loc[apartment['아파트명'] == '이천 중리 우미 린 트리쉐이드', '세대평균_주차대수'] = 1.4
apartment.loc[apartment['아파트명'] == '둔촌 현대수린나', '주차대수'] = 34
apartment.loc[apartment['아파트명'] == '서울대벤처타운역 푸르지오', '주차대수'] = 634
apartment.loc[apartment['아파트명'] == '시흥 롯데캐슬 시그니처 1블록', '주차대수'] = 1845
apartment.loc[apartment['아파트명'] == '엘리프 미아역 2단지', '주차대수'] = 248
apartment.loc[apartment['아파트명'] == '엘리프 미아역 1단지', '주차대수'] = 83
apartment.loc[apartment['아파트명'] == '왕길역 금호어울림 에듀그린', '주차대수'] = 293
apartment.loc[apartment['아파트명'] == '송도역 경남아너스빌', '주차대수'] = 255
apartment.loc[apartment['아파트명'] == '영종국제도시 A26BL 제일풍경채 디오션', '주차대수'] = 957
apartment.loc[apartment['아파트명'] == '인천 영종하늘도시 A56블록 호반써밋 스카이센트럴Ⅱ', '주차대수'] = 808
apartment.loc[apartment['아파트명'] == '시화MTV 푸르지오 디오션', '주차대수'] = 600
apartment.loc[apartment['아파트명'] == '동탄 파크릭스 A55BL', '주차대수'] = 884
apartment.loc[apartment['아파트명'] == '힐스테이트 평택 화양', '주차대수'] = 2200
apartment.loc[apartment['아파트명'] == '구리역 롯데캐슬 시그니처', '주차대수'] = 1344
apartment.loc[apartment['아파트명'] == '수원성 중흥S-클래스', '주차대수'] = 1491
apartment.loc[apartment['아파트명'] == '회천 2차 대광로제비앙 센트럴', '세대평균_주차대수'] = 1.3

In [150]:
apartment.to_csv("apartment_20230820.csv", encoding='UTF-8', index=False)